## Data Collation and Yolo Model Training
- In this colab we create a master dataset and tune a Yolov8n model

In [ ]:
# Setting up reproducibility

import random
import numpy as np
import torch
import os

SEED_VALUE = 42

random.seed(SEED_VALUE)

np.random.seed(SEED_VALUE)

torch.manual_seed(SEED_VALUE)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED_VALUE)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

os.environ['PYTHONHASHSEED'] = str(SEED_VALUE)

print(f"✅ All libraries seeded with value: {SEED_VALUE}")
print(f"   PyTorch CUDA deterministic: {torch.backends.cudnn.deterministic}")

In [2]:
from google.colab import drive
import os

drive.mount('/content/drive')

youtube_frames_path = "/content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames"

if os.path.exists(youtube_frames_path):
    print(f"✅ Google Drive mounted and YouTube frames folder found at: {youtube_frames_path}")
else:
    print(f"⚠️ ERROR: Could not find YouTube frames folder at: {youtube_frames_path}")
    print("Please double-check the path and try mounting again.")

Mounted at /content/drive
✅ Google Drive mounted and YouTube frames folder found at: /content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames


In [2]:
!pip install ultralytics -q
print("✅ Ultralytics installed.")

✅ Ultralytics installed.


### Creating Master Dataset

In [4]:
import zipfile
#unzipping filtered dataset -coco
coco_zip_path = "/content/drive/MyDrive/VisionAssist-Dataset/filtered_coco_data.zip"
unzip_destination = "temp_coco_unzipped"

if not os.path.exists(coco_zip_path):
    print(f"⚠️ ERROR: Cannot find the zip file at: {coco_zip_path}")
    print("Please make sure the file exists in your Google Drive.")
else:
    #Unzip the File
    print(f"Unzipping '{os.path.basename(coco_zip_path)}' to '{unzip_destination}'...")
    # Create the destination folder if it doesn't exist
    os.makedirs(unzip_destination, exist_ok=True)
    try:
        with zipfile.ZipFile(coco_zip_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_destination)
        print("✅ Unzipping complete.")

        potential_coco_data_folder = os.path.join(unzip_destination, "yolo_data")
        if os.path.exists(os.path.join(potential_coco_data_folder, "images")):
             actual_coco_data_folder = potential_coco_data_folder
             print(f"Found COCO data inside '{actual_coco_data_folder}'")
        elif os.path.exists(os.path.join(unzip_destination, "images")):
             actual_coco_data_folder = unzip_destination
             print(f"Found COCO data directly inside '{actual_coco_data_folder}'")
        else:
             actual_coco_data_folder = None
             print(f"⚠️ ERROR: Could not find 'images' and 'labels' folders within '{unzip_destination}'.")

    except zipfile.BadZipFile:
        print(f"⚠️ ERROR: The file '{coco_zip_path}' is not a valid zip file or is corrupted.")
        actual_coco_data_folder = None
    except Exception as e:
        print(f"⚠️ An error occurred during unzipping: {e}")
        actual_coco_data_folder = None

Unzipping 'filtered_coco_data.zip' to 'temp_coco_unzipped'...
✅ Unzipping complete.
Found COCO data directly inside 'temp_coco_unzipped'


In [5]:
import shutil
import os

if 'actual_coco_data_folder' not in locals() or actual_coco_data_folder is None:
    print("⚠️ ERROR: 'actual_coco_data_folder' is not defined. Please run Cell 1 successfully first.")
else:
    coco_data_folder = actual_coco_data_folder # Use the path found after unzipping
    youtube_frames_folder = "/content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames"
    master_dataset_folder = "master_dataset"
    master_images_folder = os.path.join(master_dataset_folder, "images")


    # create master folder
    print(f"\nCreating master images folder at '{master_images_folder}'...")
    os.makedirs(master_images_folder, exist_ok=True)

    coco_images_path = os.path.join(coco_data_folder, "images")

    # only check for image path
    if os.path.exists(coco_images_path):
        print(f"Copying images from '{coco_images_path}' to '{master_images_folder}'...")
        # Using copytree with dirs_exist_ok=True merges contents
        shutil.copytree(coco_images_path, master_images_folder, dirs_exist_ok=True)
        print("COCO images copied.")

    else:
        print(f"⚠️ ERROR: Could not find COCO 'images' folder in '{coco_data_folder}'.")

    # copy yt frames
    if os.path.exists(youtube_frames_folder):
        print(f"Copying images from '{youtube_frames_folder}' to '{master_images_folder}'...")
        copied_yt_count = 0
        for filename in os.listdir(youtube_frames_folder):
            source_file = os.path.join(youtube_frames_folder, filename)
            if os.path.isfile(source_file) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                destination_file = os.path.join(master_images_folder, filename)
                shutil.copy2(source_file, destination_file)
                copied_yt_count += 1
        print(f"Copied {copied_yt_count} YouTube frame images.")
    else:
         print(f"⚠️ Skipping YouTube frames copy as folder '{youtube_frames_folder}' was not found.")


    # verification
    print("\n--- Verification ---")
    try:
        num_master_images = len(os.listdir(master_images_folder))
        print(f"Total images in '{master_images_folder}': {num_master_images}")
        print("\n✅ Master dataset folder with images created successfully.")
        print(f"You can now point your YOLOv8 annotation script to the '{master_images_folder}'.")
        print(f"Make sure your annotation script saves the *new* label files into a separate 'labels' folder (e.g., 'master_dataset/labels').")
    except Exception as e:
        print(f"⚠️ Verification failed: {e}")


Creating master images folder at 'master_dataset/images'...
Copying images from 'temp_coco_unzipped/images' to 'master_dataset/images'...
COCO images copied.
Copying images from '/content/drive/MyDrive/VisionAssist-Dataset/Yt Frames /Example_4_frames' to 'master_dataset/images'...
Copied 2138 YouTube frame images.

--- Verification ---
Total images in 'master_dataset/images': 7138

✅ Master dataset folder with images created successfully.
You can now point your YOLOv8 annotation script to the 'master_dataset/images'.
Make sure your annotation script saves the *new* label files into a separate 'labels' folder (e.g., 'master_dataset/labels').


In [6]:
# Cell for Annotation

!pip install ultralytics -q
import torch
from ultralytics import YOLO
import os
from tqdm import tqdm # Progress bar

print("✅ Ultralytics installed.")

MASTER_IMAGES_FOLDER = "/content/master_dataset/images"
MASTER_LABELS_FOLDER = "/content/master_dataset/labels"


os.makedirs(MASTER_LABELS_FOLDER, exist_ok=True)
print(f"Labels will be saved to: {MASTER_LABELS_FOLDER}")


ANNOTATION_MODEL_NAME = 'yolov8n.pt'
print(f"Loading annotation model: {ANNOTATION_MODEL_NAME}...")
annotation_model = YOLO(ANNOTATION_MODEL_NAME)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
annotation_model.to(DEVICE)
print(f"Model loaded successfully on {DEVICE}.")


print("Will save labels for ALL detected COCO classes (0-79) using their original IDs.")


try:
    # Gets all image files from the master images folder
    image_files = [f for f in os.listdir(MASTER_IMAGES_FOLDER) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    print(f"\nFound {len(image_files)} total images to annotate in {MASTER_IMAGES_FOLDER}")
except FileNotFoundError:
    print(f"--- ERROR ---")
    print(f"Could not find the image folder: {MASTER_IMAGES_FOLDER}")
    print("Please ensure the master_dataset folder was created correctly.")
    image_files = [] # Prevent crash


annotation_count = 0 # Counter for images where labels were successfully generated

if image_files: # Only proceed if images were found
    # Loop through every image file with a progress bar
    for image_name in tqdm(image_files, desc="Annotating ALL Images"):
        image_path = os.path.join(MASTER_IMAGES_FOLDER, image_name)

        # Define the path where the label file should be saved
        label_name = os.path.splitext(image_name)[0] + ".txt"
        label_path = os.path.join(MASTER_LABELS_FOLDER, label_name)


        # Predicts objects in the current image using the loaded model
        results = annotation_model.predict(image_path, save=False, verbose=False, conf=0.25)

        #  Convert Results to YOLO TXT Format and Save
        # Checks if the prediction found any objects
        if results and len(results[0].boxes) > 0:
            lines_written = 0
            # Opens the corresponding label file for writing (overwrites if it exists)
            with open(label_path, 'w') as f:
                # Iterates through each detected object (box)
                for box in results[0].boxes:
                    # Gets the original COCO class ID (0-79)
                    coco_class_id = int(box.cls)

                    # Gets normalized coordinates [center_x, center_y, width, height]
                    x_center, y_center, width, height = box.xywhn[0].tolist()

                    # Writes the label line using the original COCO ID
                    f.write(f"{coco_class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
                    lines_written += 1 # Count line written

            # If we wrote labels to the file, increment the success counter
            if lines_written > 0:
                 annotation_count += 1
            # If no objects were detected (lines_written == 0),
            # but an empty file was created, remove the empty file.
            elif os.path.exists(label_path):
                 try: # Add error handling just in case
                     if os.path.getsize(label_path) == 0:
                          os.remove(label_path)
                 except OSError:
                      pass # Ignore error if file is already gone etc.

        # else: # No detections made by the model for this image
            # No label file needed or created.

    # final summary
    print(f"\n--- Annotation Summary ---")
    print(f"Processed {len(image_files)} images.")
    print(f"Generated or overwrote labels for {annotation_count} images.")
    print(f"Labels were saved using original COCO class IDs (0-79).")
    print(f"All labels are now saved in: {MASTER_LABELS_FOLDER}")
else:
    # Message if the image folder was empty or not found
    print("No images found to process.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.3 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Ultralytics installed.
Labels will be saved to: /content/master_dataset/labels
Loading annotation model: yolov8n.pt...
Model loaded successfully on cuda.
Will save labels for ALL detected COCO classes (0-79) using their original IDs.

Found 7138 total images to annotate in /content/master_dataset/images


Annotating ALL Images: 100%|██████████| 7138/7138 [02:15<00:00, 52.70it/s]


--- Annotation Summary ---
Processed 7138 images.
Generated or overwrote labels for 7102 images.
Labels were saved using original COCO class IDs (0-79).
All labels are now saved in: /content/master_dataset/labels


In [7]:
!pip install supervision numpy opencv-python lap

print("✅ Necessary libraries for ByteTrack and tracking installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.3 MB/s eta 0:00:00
✅ Necessary libraries for ByteTrack and tracking installed.


### Splitting the dataset

In [8]:
import os
import random
import shutil
from tqdm import tqdm

# Configuration
MASTER_DATASET_FOLDER = "master_dataset" # Folder containing 'images' and 'labels'
OUTPUT_FOLDER = "split_dataset"        # New folder where 'train', 'valid', 'test' will be created
TRAIN_RATIO = 0.70                     # 70% for training
VALID_RATIO = 0.20                     # 20% for validation
TEST_RATIO = 0.10                      # 10% for testing

# Sanity Check
if abs(TRAIN_RATIO + VALID_RATIO + TEST_RATIO - 1.0) > 1e-6:
    print("⚠️ ERROR: Ratios must add up to 1.0")
else:
    # Define paths
    images_input_dir = os.path.join(MASTER_DATASET_FOLDER, "images")
    labels_input_dir = os.path.join(MASTER_DATASET_FOLDER, "labels")

    train_images_dir = os.path.join(OUTPUT_FOLDER, "train", "images")
    train_labels_dir = os.path.join(OUTPUT_FOLDER, "train", "labels")
    valid_images_dir = os.path.join(OUTPUT_FOLDER, "valid", "images")
    valid_labels_dir = os.path.join(OUTPUT_FOLDER, "valid", "labels")
    test_images_dir = os.path.join(OUTPUT_FOLDER, "test", "images")
    test_labels_dir = os.path.join(OUTPUT_FOLDER, "test", "labels")

    # Create output directories
    print(f"Creating output directories in '{OUTPUT_FOLDER}'...")
    os.makedirs(train_images_dir, exist_ok=True)
    os.makedirs(train_labels_dir, exist_ok=True)
    os.makedirs(valid_images_dir, exist_ok=True)
    os.makedirs(valid_labels_dir, exist_ok=True)
    os.makedirs(test_images_dir, exist_ok=True)
    os.makedirs(test_labels_dir, exist_ok=True)

    # Get List of All Image Files
    try:
        all_images = [f for f in os.listdir(images_input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        if not all_images:
             print(f"⚠️ ERROR: No images found in '{images_input_dir}'.")
        else:
             print(f"Found {len(all_images)} images in '{images_input_dir}'.")
             # shuffle the images
             random.shuffle(all_images)

             # Calculate Split Indices
             total_images = len(all_images)
             train_end_idx = int(total_images * TRAIN_RATIO)
             valid_end_idx = train_end_idx + int(total_images * VALID_RATIO)
             # The rest go to test

             # Define Splits
             train_files = all_images[:train_end_idx]
             valid_files = all_images[train_end_idx:valid_end_idx]
             test_files = all_images[valid_end_idx:]

             print(f"Splitting into: {len(train_files)} Train, {len(valid_files)} Valid, {len(test_files)} Test")

             # Function to Copy Files
             def copy_files(file_list, source_img_dir, source_lbl_dir, dest_img_dir, dest_lbl_dir):
                 copied_count = 0
                 for filename in tqdm(file_list, desc=f"Copying to {os.path.basename(dest_img_dir)}"):
                     # Copy Image
                     source_image_path = os.path.join(source_img_dir, filename)
                     dest_image_path = os.path.join(dest_img_dir, filename)
                     if os.path.exists(source_image_path):
                         shutil.copy2(source_image_path, dest_image_path)

                         # Copy Corresponding Label (if it exists)
                         label_filename = os.path.splitext(filename)[0] + ".txt"
                         source_label_path = os.path.join(source_lbl_dir, label_filename)
                         dest_label_path = os.path.join(dest_lbl_dir, label_filename)
                         if os.path.exists(source_label_path):
                             shutil.copy2(source_label_path, dest_label_path)
                         # else:
                         #     print(f"Warning: Label file not found for {filename}") # Optional warning
                         copied_count += 1
                 return copied_count

             # Perform the Copying
             print("\nCopying training files...")
             train_copied = copy_files(train_files, images_input_dir, labels_input_dir, train_images_dir, train_labels_dir)
             print(f"Copied {train_copied} training images and labels.")

             print("\nCopying validation files...")
             valid_copied = copy_files(valid_files, images_input_dir, labels_input_dir, valid_images_dir, valid_labels_dir)
             print(f"Copied {valid_copied} validation images and labels.")

             print("\nCopying test files...")
             test_copied = copy_files(test_files, images_input_dir, labels_input_dir, test_images_dir, test_labels_dir)
             print(f"Copied {test_copied} test images and labels.")

             print("\n✅ Dataset splitting complete!")
             print(f"Split dataset is ready in the '{OUTPUT_FOLDER}' folder.")

    except FileNotFoundError:
        print(f"--- ERROR ---")
        print(f"Could not find the image folder: {images_input_dir}")
        print("Please ensure the master_dataset folder exists and contains images.")
    except Exception as e:
        print(f"--- An unexpected error occurred: {e} ---")

Creating output directories in 'split_dataset'...
Found 7138 images in 'master_dataset/images'.
Splitting into: 4996 Train, 1427 Valid, 715 Test

Copying training files...


Copying to images: 100%|██████████| 4996/4996 [00:01<00:00, 2781.33it/s]


Copied 4996 training images and labels.

Copying validation files...


Copying to images: 100%|██████████| 1427/1427 [00:00<00:00, 2589.73it/s]


Copied 1427 validation images and labels.

Copying test files...


Copying to images: 100%|██████████| 715/715 [00:00<00:00, 810.98it/s] 

Copied 715 test images and labels.

✅ Dataset splitting complete!
Split dataset is ready in the 'split_dataset' folder.


In [9]:
# mounting this in drive.

import shutil
import os

temp_dataset_folder = '/content/split_dataset'
permanent_dataset_folder = '/content/drive/My Drive/VisionAssist-Dataset/split_dataset'

if os.path.exists(permanent_dataset_folder):
    print(f"Dataset already exists at: {permanent_dataset_folder}")
else:
    print(f"Copying temporary dataset from {temp_dataset_folder} to {permanent_dataset_folder}...")
    try:
        shutil.copytree(temp_dataset_folder, permanent_dataset_folder)
        print("✅ Successfully created permanent dataset in Google Drive.")
    except Exception as e:
        print(f"⚠️ Error copying dataset: {e}")

Dataset already exists at: /content/drive/My Drive/VisionAssist-Dataset/split_dataset


### Creating a data.yaml file


In [10]:
%%writefile coco_custom_data.yaml
# YAML configuration file for YOLOv8 training

# Dataset paths: Relative paths to the split dataset folder
# Assumes this yaml file is in /content/ and the data is in /content/split_dataset/
train: ./split_dataset/train/images
val: ./split_dataset/valid/images
test: ./split_dataset/test/images # Optional: path to test images

# Number of classes
# Must be 80 since we used original COCO IDs during annotation
nc: 80

# Class names
# Must be the standard 80 COCO class names in the correct order (0-79)
names:
  0: person
  1: bicycle
  2: car
  3: motorcycle
  4: airplane
  5: bus
  6: train
  7: truck
  8: boat
  9: traffic light
  10: fire hydrant
  11: stop sign
  12: parking meter
  13: bench
  14: bird
  15: cat
  16: dog
  17: horse
  18: sheep
  19: cow
  20: elephant
  21: bear
  22: zebra
  23: giraffe
  24: backpack
  25: umbrella
  26: handbag
  27: tie
  28: suitcase
  29: frisbee
  30: skis
  31: snowboard
  32: sports ball
  33: kite
  34: baseball bat
  35: baseball glove
  36: skateboard
  37: surfboard
  38: tennis racket
  39: bottle
  40: wine glass
  41: cup
  42: fork
  43: knife
  44: spoon
  45: bowl
  46: banana
  47: apple
  48: sandwich
  49: orange
  50: broccoli
  51: carrot
  52: hot dog
  53: pizza
  54: donut
  55: cake
  56: chair
  57: couch
  58: potted plant
  59: bed
  60: dining table
  61: toilet
  62: tv
  63: laptop
  64: mouse
  65: remote
  66: keyboard
  67: cell phone
  68: microwave
  69: oven
  70: toaster
  71: sink
  72: refrigerator
  73: book
  74: clock
  75: vase
  76: scissors
  77: teddy bear
  78: hair drier
  79: toothbrush

Overwriting coco_custom_data.yaml


In [11]:
import shutil
import os
from google.colab import drive

colab_file_path = 'coco_custom_data.yaml'

# Path where you want to save the file in Google Drive

drive_save_folder = '/content/drive/My Drive/VisionAssist-Dataset/'
drive_save_path = os.path.join(drive_save_folder, os.path.basename(colab_file_path))

# --- Mount Drive (if not already mounted) ---
try:
    if not os.path.exists('/content/drive/My Drive'):
         print("Mounting Google Drive...")
         drive.mount('/content/drive')
    else:
         print("Google Drive already mounted.")
except Exception as e:
    print(f"Error mounting drive: {e}")

# --- Check if File Exists in Colab ---
if not os.path.exists(colab_file_path):
    print(f"--- ERROR: File not found in Colab at '{colab_file_path}' ---")
    print("Please ensure the 'coco_custom_data.yaml' file was created successfully.")
else:
    # --- Create Save Directory in Drive ---
    print(f"Ensuring save directory exists: {drive_save_folder}")
    os.makedirs(drive_save_folder, exist_ok=True)

    # --- Copy the File ---
    print(f"Copying '{os.path.basename(colab_file_path)}' to '{drive_save_path}'...")
    try:
        shutil.copy(colab_file_path, drive_save_path)
        print("✅ File successfully saved to Google Drive.")
    except Exception as e:
        print(f"--- ERROR saving file to Drive: {e} ---")

Google Drive already mounted.
Ensuring save directory exists: /content/drive/My Drive/VisionAssist-Dataset/
Copying 'coco_custom_data.yaml' to '/content/drive/My Drive/VisionAssist-Dataset/coco_custom_data.yaml'...
✅ File successfully saved to Google Drive.


### Fine Tuning YOLOV8N

In [14]:
import optuna
from ultralytics import YOLO
import time
import os

# Constants
WORKERS = max(2, os.cpu_count() // 2)
N_TRIALS = 15
EPOCHS_PER_TRIAL = 15
PATIENCE = 3

print(f"--- Using global SEED_VALUE: {SEED_VALUE} ---")

def objective(trial):
    try:
         # learning rate
        lr0 = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
        # optimization
        # Weight decay helps prevent overfitting.
        weight_decay = trial.suggest_float("weight_decay", 5e-5, 5e-3, log=True)
        # testing different optimizers
        optimizer = trial.suggest_categorical("optimizer", ["AdamW", "SGD"])
        degrees = trial.suggest_float("degrees", 0.0, 5.0)
        # data augmentation
        translate = trial.suggest_float("translate", 0.0, 0.1)  # less rotation
        scale = trial.suggest_float("scale", 0.7, 0.9)    # less translation
        hsv_s = trial.suggest_float("hsv_s", 0.4, 0.7)     # similar scale
        hsv_v = trial.suggest_float("hsv_v", 0.4, 0.7)
        fliplr = trial.suggest_float("fliplr", 0.0, 0.5)

        # create and train the model
        model = YOLO("yolov8n.pt")

        # Start training
        model.train(
            data="coco_custom_data.yaml",
            freeze=True,
            epochs=EPOCHS_PER_TRIAL,
            patience=PATIENCE,
            imgsz=640,
            batch=64,
            workers=WORKERS,
            device=0,
            project="optuna_yolo_fine_tune",
            name=f"trial_{trial.number}",

            # Hyperparameters from Optuna
            optimizer=optimizer,
            lr0=lr0,
            weight_decay=weight_decay,
            degrees=degrees,
            translate=translate,
            scale=scale,
            hsv_s=hsv_s,
            hsv_v=hsv_v,
            fliplr=fliplr,

            verbose=False,
            amp=True,
            plots=False,


            seed=SEED_VALUE,
            deterministic=True

        )

        metrics = getattr(model, "metrics", {})
        mAP50 = getattr(metrics, "results_dict", {}).get("metrics/mAP50(B)", 0.0)

        return mAP50

    except Exception as e:
        print(f"Trial {trial.number} failed with error: {e}")
        return 0.0


# Progress tracking
start_time = time.time()

def callback(study, trial):
    elapsed = time.time() - start_time
    done = len(study.trials)
    avg_time = elapsed / done if done > 0 else 0
    remaining = (N_TRIALS - done) * avg_time
    print(f"\n🧠 Trial {done}/{N_TRIALS} completed")
    print(f"   ⏱ Elapsed: {elapsed/60:.2f} min | Est. remaining: {remaining/60:.2f} min")
    print(f"   📈 Best mAP50 so far: {study.best_value:.4f}")



# Define a seeded sampler for Optuna
sampler = optuna.samplers.TPESampler(seed=SEED_VALUE)

print(f"Starting Optuna optimization... Searching for {N_TRIALS} trials with seed {SEED_VALUE}.")


study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=N_TRIALS, callbacks=[callback])

# final report
total_time = (time.time() - start_time) / 60
print("\n" + "="*30)
print("🏆 BEST TRIAL SUMMARY 🏆")
print("="*30)
print(f"   🕒 Total time: {total_time:.2f} min")
print(f"   📊 Best mAP50: {study.best_trial.value:.4f}")
print("   ⚙️ Best Params:")
for key, value in study.best_trial.params.items():
    print(f"      {key}: {value}")

[I 2025-11-04 15:39:05,585] A new study created in memory with name: no-name-b324e2dd-162c-4078-a881-cc4b4bcb0c69


Starting Optuna optimization... Searching for 15 trials.
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco_custom_data.yaml, degrees=2.02234428260504, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.14726144774341782, flipud=0.0, format=torchscript, fraction=1.0, freeze=True, half=False, hsv_h=0.015, hsv_s=0.6033093626797703, hsv_v=0.6444198248957093, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00012889634912898807, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=trial_0, nbs=6

[I 2025-11-04 15:45:29,721] Trial 0 finished with value: 0.8820322074102609 and parameters: {'lr0': 0.00012889634912898807, 'weight_decay': 0.0005177406727578624, 'optimizer': 'SGD', 'degrees': 2.02234428260504, 'translate': 0.09068219543990846, 'scale': 0.8008608451780083, 'hsv_s': 0.6033093626797703, 'hsv_v': 0.6444198248957093, 'fliplr': 0.14726144774341782}. Best is trial 0 with value: 0.8820322074102609.



🧠 Trial 1/15 completed
   ⏱ Elapsed: 6.40 min | Est. remaining: 89.63 min
   📈 Best mAP50 so far: 0.8820
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco_custom_data.yaml, degrees=3.7589273841096356, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.026195651928720587, flipud=0.0, format=torchscript, fraction=1.0, freeze=True, half=False, hsv_h=0.015, hsv_s=0.5549357382886335, hsv_v=0.45552370718613133, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=1.1715560211053696e-05, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.93

[I 2025-11-04 15:54:26,862] Trial 1 finished with value: 0.8817609013423634 and parameters: {'lr0': 1.1715560211053696e-05, 'weight_decay': 0.0002346732908940307, 'optimizer': 'SGD', 'degrees': 3.7589273841096356, 'translate': 0.05667051099823243, 'scale': 0.8822242326607416, 'hsv_s': 0.5549357382886335, 'hsv_v': 0.45552370718613133, 'fliplr': 0.026195651928720587}. Best is trial 0 with value: 0.8820322074102609.


...


🧠 Trial 13/15 completed
   ⏱ Elapsed: 99.25 min | Est. remaining: 15.27 min
   📈 Best mAP50 so far: 0.8881
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco_custom_data.yaml, degrees=0.13644917499562953, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.3839220549560274, flipud=0.0, format=torchscript, fraction=1.0, freeze=True, half=False, hsv_h=0.015, hsv_s=0.6905748958175771, hsv_v=0.4044942134790513, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00027926356598975205, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.93

[I 2025-11-04 17:26:16,785] Trial 13 finished with value: 0.888064046351472 and parameters: {'lr0': 0.00027926356598975205, 'weight_decay': 0.001727823848693769, 'optimizer': 'SGD', 'degrees': 0.13644917499562953, 'translate': 0.000904673616974576, 'scale': 0.8326561730448047, 'hsv_s': 0.6905748958175771, 'hsv_v': 0.4044942134790513, 'fliplr': 0.3839220549560274}. Best is trial 11 with value: 0.8881091835395264.



🧠 Trial 14/15 completed
   ⏱ Elapsed: 107.19 min | Est. remaining: 7.66 min
   📈 Best mAP50 so far: 0.8881
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=coco_custom_data.yaml, degrees=0.962848924344557, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.3821475704194012, flipud=0.0, format=torchscript, fraction=1.0, freeze=True, half=False, hsv_h=0.015, hsv_s=0.6939828894885501, hsv_v=0.40737628858054126, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00039254651878206415, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937

[I 2025-11-04 17:32:43,976] Trial 14 finished with value: 0.8891296543012678 and parameters: {'lr0': 0.00039254651878206415, 'weight_decay': 0.002112269394178678, 'optimizer': 'SGD', 'degrees': 0.962848924344557, 'translate': 0.014265842978098032, 'scale': 0.7691910594505441, 'hsv_s': 0.6939828894885501, 'hsv_v': 0.40737628858054126, 'fliplr': 0.3821475704194012}. Best is trial 14 with value: 0.8891296543012678.



🧠 Trial 15/15 completed
   ⏱ Elapsed: 113.64 min | Est. remaining: 0.00 min
   📈 Best mAP50 so far: 0.8891

🏆 BEST TRIAL SUMMARY 🏆
   🕒 Total time: 113.64 min
   📊 Best mAP50: 0.8891
   ⚙️ Best Params:
      lr0: 0.00039254651878206415
      weight_decay: 0.002112269394178678
      optimizer: SGD
      degrees: 0.962848924344557
      translate: 0.014265842978098032
      scale: 0.7691910594505441
      hsv_s: 0.6939828894885501
      hsv_v: 0.40737628858054126
      fliplr: 0.3821475704194012


In [13]:
!pip install optuna -q
print("✅ Optuna installed.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 30.5 MB/s eta 0:00:00
✅ Optuna installed.


### Saving the best model

In [16]:
import os
import shutil
from google.colab import drive

PROJECT_NAME = "optuna_yolo_fine_tune"
DRIVE_FOLDER = '/content/drive/My Drive/VisionAssist-Models/'

print("--- Starting Model Save ---")

try:
    #  Get Best Trial Info
    best_trial_num = study.best_trial.number
    best_trial_value = study.best_trial.value
    print(f"✅ Found best trial: Trial #{best_trial_num} with mAP50: {best_trial_value:.4f}")

    #  Define Paths
    trained_model_path_colab = os.path.join(PROJECT_NAME, f"trial_{best_trial_num}", "weights", "best.pt")
    model_save_name = f'yolov8n_optuna_trial{best_trial_num}_map{best_trial_value:.4f}.pt'
    drive_save_path = os.path.join(DRIVE_FOLDER, model_save_name)

    # Mount Drive
    if not os.path.exists('/content/drive/My Drive'):
         print("Mounting Google Drive...")
         drive.mount('/content/drive')
    else:
         print("Google Drive already mounted.")

    # Check if Model File Exists
    if not os.path.exists(trained_model_path_colab):
        print(f"--- ⛔ ERROR: Model file not found at '{trained_model_path_colab}' ---")
        print("Please check the 'PROJECT_NAME' variable.")
    else:
        #  Create Save Directory & Copy
        print(f"Ensuring save directory exists: {DRIVE_FOLDER}")
        os.makedirs(DRIVE_FOLDER, exist_ok=True)

        print(f"Copying '{trained_model_path_colab}' to '{drive_save_path}'...")
        shutil.copy(trained_model_path_colab, drive_save_path)

        print("\n" + "="*30)
        print("🏆 SUCCESS! Model saved to Google Drive. 🏆")
        print(f"   Saved as: {model_save_name}")
        print("="*30)

except Exception as e:
    print(f"--- ⛔ AN ERROR OCCURRED: {e} ---")
    print("If you see 'study is not defined', you may have disconnected from the Colab runtime.")

--- Starting Model Save ---
✅ Found best trial: Trial #14 with mAP50: 0.8891
Google Drive already mounted.
Ensuring save directory exists: /content/drive/My Drive/VisionAssist-Models/
Copying 'optuna_yolo_fine_tune/trial_14/weights/best.pt' to '/content/drive/My Drive/VisionAssist-Models/yolov8n_optuna_trial14_map0.8891.pt'...

🏆 SUCCESS! Model saved to Google Drive. 🏆
   Saved as: yolov8n_optuna_trial14_map0.8891.pt
